In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [2]:
required_stocks = {
    "albemarle": "alb",
    "ganfeng": "gnenf",
    "livent": "lthm",
    "lithium americas": "lac",
    "lg chem": "051910",
    "toshiba corp": "tosyy",
    "panasonic": "pcrfy",
    "samsung": "005930"
}

In [3]:
DATA_SOURCE_NAME = "Kaggle"
DATA_DIR = os.path.join("NewsData", "Kaggle")

In [4]:
all_timestamps = []
all_headlines = []
all_tickers = []

In [5]:
def process_dataset(required_stocks, file_path, timestamp_header, headline_header, ticker_header=None, datetime_str_format="%Y-%m-%d %H:%M:%S%z"):
    dataset = pd.read_csv(file_path)
    print("initial size:", dataset.shape[0])
    
    timestamps, headlines, tickers = [], [], [] 
    
    def process_date_field(timestamp):
        try:
            timestamp = str(timestamp)
            return datetime.strptime(timestamp, datetime_str_format)
        except:
            return None
    
    # Convert header and ticker to lowercase
    dataset[headline_header] = dataset[headline_header].str.lower()
    if ticker_header:
        dataset[ticker_header] = dataset[ticker_header].str.lower()
    
    # Convert date fields to proper datetime object
    dataset[timestamp_header] = dataset[timestamp_header].apply(process_date_field)
    dataset = dataset[dataset[timestamp_header].notnull()]
    print('after filtering date field:', dataset.shape[0])
    
    # Process dataset
    for stock_name, stock_ticker in required_stocks.items():
        if ticker_header:
            filtered_df = dataset[
                dataset[headline_header].str.contains(stock_name)
                | dataset[headline_header].str.contains(" " + stock_ticker)
                | dataset[headline_header].str.contains(stock_ticker + " ")
                | dataset[ticker_header].str.contains(stock_ticker)
            ]
        else:
            filtered_df = dataset[
                dataset[headline_header].str.contains(stock_name)
                | dataset[headline_header].str.contains(" " + stock_ticker)
                | dataset[headline_header].str.contains(stock_ticker + " ")
            ]
        tickers.extend([stock_ticker]*filtered_df.shape[0])

    timestamps = filtered_df[timestamp_header].tolist()
    headlines = filtered_df[headline_header].tolist()
    
    print("final size:", len(timestamps))
    
    return timestamps, headlines, tickers

### Process dataset 1

In [6]:
FILENAME = "analyst_ratings_processed.csv"
timestamps, headlines, tickers = process_dataset(
    required_stocks = required_stocks,
    file_path = os.path.join(DATA_DIR, FILENAME),
    timestamp_header = 'date',
    headline_header = 'title',
    ticker_header = 'stock',
    datetime_str_format = '%Y-%m-%d %H:%M:%S%z'
)
all_timestamps.extend(timestamps)
all_headlines.extend(headlines)
all_tickers.extend(tickers)

initial size: 1400469
after filtering date field: 1397891
final size: 749


### Process dataset 2

In [7]:
FILENAME = "abcnews-date-text.csv"
timestamps, headlines, tickers = process_dataset(
    required_stocks = required_stocks,
    file_path = os.path.join(DATA_DIR, FILENAME),
    timestamp_header = 'publish_date',
    headline_header = 'headline_text',
    ticker_header = None,
    datetime_str_format = '%Y%m%d'
)
all_timestamps.extend(timestamps)
all_headlines.extend(headlines)
all_tickers.extend(tickers)

initial size: 1244184
after filtering date field: 1244184
final size: 93


### Process dataset 3

In [8]:
FILENAME = "RedditNews.csv"
timestamps, headlines, tickers = process_dataset(
    required_stocks = required_stocks,
    file_path = os.path.join(DATA_DIR, FILENAME),
    timestamp_header = 'Date',
    headline_header = 'News',
    ticker_header = None,
    datetime_str_format = '%Y-%m-%d'
)
all_timestamps.extend(timestamps)
all_headlines.extend(headlines)
all_tickers.extend(tickers)

initial size: 73608
after filtering date field: 73608
final size: 22


### Process dataset 4

In [9]:
FILENAME = "us_equities_news_dataset.csv"
timestamps, headlines, tickers = process_dataset(
    required_stocks = required_stocks,
    file_path = os.path.join(DATA_DIR, FILENAME),
    timestamp_header = 'release_date',
    headline_header = 'title',
    ticker_header = 'ticker',
    datetime_str_format = '%Y-%m-%d'
)
all_timestamps.extend(timestamps)
all_headlines.extend(headlines)
all_tickers.extend(tickers)

initial size: 221513
after filtering date field: 221513
final size: 459


### Process dataset 5

In [10]:
FILENAME = "raw_partner_headlines.csv"
timestamps, headlines, tickers = process_dataset(
    required_stocks = required_stocks,
    file_path = os.path.join(DATA_DIR, FILENAME),
    timestamp_header = 'date',
    headline_header = 'headline',
    ticker_header = 'stock',
    datetime_str_format = '%Y-%m-%d %H:%M:%S'
)
all_timestamps.extend(timestamps)
all_headlines.extend(headlines)
all_tickers.extend(tickers)

initial size: 1845559
after filtering date field: 1845559
final size: 747


### Create final dataset

In [18]:
TIMESTAMP_HEADER = 'date'
HEADLINE_HEADER = 'headline'
TICKER_HEADER = 'ticker'
SOURCE_HEADER = 'source'
INDEX_HEADER = 'idx'

n = len(all_timestamps)
str_timestamps = [timestamp.strftime('%Y-%m-%dT%H:%M:%SZ') for timestamp in all_timestamps]

final_df = pd.DataFrame(
    list(zip(
        all_headlines,
        all_timestamps,
        all_tickers,
        [DATA_SOURCE_NAME]*n)),
    columns = [HEADLINE_HEADER, TIMESTAMP_HEADER, TICKER_HEADER, SOURCE_HEADER])

# Remove duplicate rows
print("Before deduplication", final_df.shape)
final_df = final_df.drop_duplicates()
print("After deduplication", final_df.shape)

Before deduplication (2070, 4)
After deduplication (1489, 4)


In [19]:
final_df.to_csv('final_kaggle_data.csv', index_label = INDEX_HEADER)